In [ ]:
!pip install opendatasets

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import time

import opendatasets as od

ModuleNotFoundError: No module named 'opendatasets'

In [ ]:
od.download("https://www.kaggle.com/datasets/dineshpiyasamara/cats-and-dogs-for-classification")

In [ ]:
BATCH_SIZE=32
image_size=(128,128)

In [ ]:
train_data_dir="/content/cats-and-dogs-for-classification/cats_dogs/train"
test_data_dir="/content/cats-and-dogs-for-classification/cats_dogs/test"

In [ ]:
train_data=tf.keras.utils.image_dataset_from_directory(train_data_dir,batch_size=BATCH_SIZE,
                                                       image_size=image_size,
                                                       subset='training',
                                                       validation_split=0.1,
                                                       seed=42)
validation_data=tf.keras.utils.image_dataset_from_directory(train_data_dir,batch_size=BATCH_SIZE,
                                                       image_size=image_size,
                                                       subset='validation',
                                                       validation_split=0.1,
                                                       seed=42)
test_data=tf.keras.utils.image_dataset_from_directory(test_data_dir,batch_size=BATCH_SIZE,
                                                       image_size=image_size)

In [ ]:
class_names=train_data.class_names
class_names

In [ ]:
train_data.take(1)

In [ ]:
plt.figure(figsize=(10,4))
for image,label in train_data.take(1):
  for i in range(10):
    ax=plt.subplot(2,5,i+1)
    plt.imshow(image[i].numpy().astype('uint8'))
    plt.title(class_names[label[i]])
    plt.axis('off')

In [ ]:
for image.label in train_data.take(1):
  for i in range(1):
    print(image)

In [ ]:
train_data=train_data.map(lambda x,y:(x/255,y))
validation_data=validation_data.map(lambda x,y:(x/255,y))
test_data=test_data.map(lambda x,y:(x/255,y))

In [ ]:
for image.label in train_data.take(1):
  for i in range(1):
    print(image)

In [ ]:
data_augmentation=tf.keras.Sequential(
    [
    tf.keras.layers.RandomFlip('horizontal',input_shape=(128,128,3)),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
    ]
)

In [ ]:
model=tf.keras.models.Sequential()

model.add(data_augmentation)

model.add(tf.keras.layers.Conv2D(32,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(64,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Conv2D(128,kernel_size=3,activation='relu'))
model.add(tf.keras.layers.MaxPooling2D())

model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.BatchNormalization())

model.add(tf.keras.layers.Flatten())

model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(32,activation='relu'))

model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])


In [ ]:
start_time=time.time()
history=model.fit(train_data,epochs=20,validation_data=validation_data)
end_time=time.time()

In [ ]:
history.history

In [ ]:
fig=plt.figure()
plt.plot(history.history['accuracy'],color='red',label='accuracy')
plt.plot(history.history['val_accuracy'],color='yellow',label='val_accuracy')
fig.suptitle('Accurcy',fontsize=20)
plt.legend()
plt.show()

In [ ]:
fig=plt.figure()
plt.plot(history.history['loss'],color='red',label='loss')
plt.plot(history.history['val_loss'],color='yellow',label='val_loss')
fig.suptitle('Loss',fontsize=20)
plt.legend()
plt.show()

In [ ]:
precision=tf.keras.metrics.Precision()
recall=tf.keras.metrics.Recall()
accuracy=tf.keras.metrics.BinaryAccuracy()

In [ ]:
for batch in test_data.as_numpy_iterator():
  X,y=batch
  yhat=model.predict(X)
  precision.update_state(y,yhat)
  recall.update_state(y,yhat)
  accuracy.update_state(y,yhat)

In [ ]:
precision.result()

In [ ]:
recall.result()

In [ ]:
accuracy.result()

In [ ]:
!pip install opencv-python

In [ ]:
import cv2